In [53]:
import os
import io

from azure.identity import DefaultAzureCredential
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.core.exceptions import ResourceExistsError

from dotenv import load_dotenv
import pandas as pd

In [37]:
load_dotenv('../.env')

tenant_id = os.environ['TENANT_ID']
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
container = "raw"
blob = "tourism_dataset.csv"

credential = DefaultAzureCredential(authority=f"https://login.microsoftonline.com/{tenant_id}")

RESOURCE_GROUP_NAME = "Data_Engineer"
LOCATION = "westeurope"
NSG_NAME = "saim-nsg"
VNET_NAME = "saim-nazir-vnet"
SUBNET_NAME = "saim-subnet"
IP_NAME = "saim-ip"
IP_CONFIG_NAME = "saim-ip-config"
NIC_NAME = "saim-nic"
VM_NAME = "VM-Saim"
USERNAME = "Saim"
PASSWORD = os.environ["PASSWORD"]

In [19]:
# Obtain the management object for networks
network_client = NetworkManagementClient(credential, subscription_id)
compute_client = ComputeManagementClient(credential, subscription_id)

In [7]:
# Provision the virtual network and wait for completion
poller = network_client.virtual_networks.begin_create_or_update(
    RESOURCE_GROUP_NAME,
    VNET_NAME,
    {
        "location": LOCATION,
        "address_space": {"address_prefixes": ["10.0.0.0/16"]},
    },
)

vnet_result = poller.result()

print(
    f"Provisioned virtual network {vnet_result.name} with address \
prefixes {vnet_result.address_space.address_prefixes}"
)

Provisioned virtual network saim-vnet with address prefixes ['10.0.0.0/16']


In [16]:
# Create a NSG since DTU has a policy: "Subnets should have a Network Security Group",

print(f"Creating Network Security Group {NSG_NAME}...")
nsg_params = {
    "location": LOCATION,
    "security_rules": [
        {
            "name": "allow_ssh",
            "protocol": "Tcp",
            "source_address_prefix": "*",
            "source_port_range": "*",
            "destination_address_prefix": "*",
            "destination_port_range": "22",
            "access": "Allow",
            "priority": 1000,
            "direction": "Inbound",
        }
    ],
}
nsg_poller = network_client.network_security_groups.begin_create_or_update(
    RESOURCE_GROUP_NAME,
    NSG_NAME,
    nsg_params
    
)
nsg_result = nsg_poller.result()
print(f"Network Security Group {nsg_result.name} has been created.")

Creating Network Security Group saim-nsg...
Network Security Group saim-nsg has been created.


In [17]:
print(f"Provisioning subnet {SUBNET_NAME}...")
subnet_params = {
    "address_prefix": "10.0.0.0/24",
    "service_endpoints": [{"service": "Microsoft.Storage"}],
    "network_security_group": {"id": nsg_result.id},
}
subnet_poller = network_client.subnets.begin_create_or_update(
    RESOURCE_GROUP_NAME,
    VNET_NAME,
    SUBNET_NAME,
    subnet_params
)
subnet_result = subnet_poller.result()
print(f"Provisioned subnet {subnet_result.name}")

Provisioning subnet saim-subnet...
Provisioned subnet saim-subnet


In [58]:
public_ip_params = {
    "location": LOCATION,
    "public_ip_allocation_method": "Static",
    "sku": {
        "name": "Standard"
    }
}
public_ip_poller = network_client.public_ip_addresses.begin_create_or_update(
    RESOURCE_GROUP_NAME,
    IP_NAME,
    public_ip_params
)
public_ip_result = public_ip_poller.result()
print(f"Public IP address {public_ip_result.name} has been created.")

Public IP address saim-ip has been created.


In [59]:
# Step 4: Provision the network interface
print(f"Provisioning network interface {NIC_NAME}...")
nic_params = {
    "location": LOCATION,
    "ip_configurations": [
        {
            "name": "ipconfig1",
            "subnet": {"id": subnet_result.id},
            "public_ip_address": {"id": public_ip_result.id}
        }
    ],
}
nic_poller = network_client.network_interfaces.begin_create_or_update(
    RESOURCE_GROUP_NAME,
    NIC_NAME,
    nic_params
)
nic_result = nic_poller.result()
print(f"Provisioned network interface {nic_result.name}")

Provisioning network interface saim-nic...
Provisioned network interface saim-nic


In [26]:
# Step 5: Provision the virtual machine
print(f"Provisioning virtual machine {VM_NAME}; this operation might take a few minutes.")
vm_params = {
    "location": LOCATION,
    "storage_profile": {
        "image_reference": {
            "publisher": "Canonical",
            "offer": "UbuntuServer",
            "sku": "18.04-LTS",
            "version": "latest",
        }
    },
    "hardware_profile": {"vm_size": "Standard_DS1_v2"},
    "os_profile": {
        "computer_name": VM_NAME,
        "admin_username": USERNAME,
        "admin_password": PASSWORD,
    },
    "network_profile": {
        "network_interfaces": [
            {
                "id": nic_result.id,
            }
        ]
    },
}
vm_poller = compute_client.virtual_machines.begin_create_or_update(
    RESOURCE_GROUP_NAME,
    VM_NAME,
    vm_params
)
vm_result = vm_poller.result()
print(f"Provisioned virtual machine {vm_result.name}")

Provisioning virtual machine VM-Saim; this operation might take a few minutes.
Provisioned virtual machine VM-Saim


## Step 2: Read Data from Azure Storage Account

In [30]:
from azure.storage.blob import BlobServiceClient

account_name = "dataengineerv1"

storage_account_url = f"https://{account_name}.blob.core.windows.net/"


blob_service_client = BlobServiceClient(account_url=storage_account_url, credential=credential)



In [39]:
blob_client = blob_service_client.get_blob_client(container=container, blob=blob)

# Download the CSV file as bytes
downloaded_blob = blob_client.download_blob()

csv_content = downloaded_blob.content_as_text()

# Use io.StringIO to read the CSV content as a DataFrame
df = pd.read_csv(io.StringIO(csv_content))

In [40]:
df.head()

,Location,Country,Category,Visitors,Rating,Revenue,Accommodation_Available
0,kuBZRkVsAR,India,Nature,948853,1.32,84388.38,Yes
1,aHKUXhjzTo,USA,Historical,813627,2.01,802625.60,No
2,dlrdYtJFTA,Brazil,Nature,508673,1.42,338777.11,Yes
3,DxmlzdGkHK,Brazil,Historical,623329,1.09,295183.60,Yes
4,WJCCQlepnz,France,Cultural,124867,1.43,547893.24,No


## Step 3: Perform Data Analysis

In [45]:
# Group the data by 'Country' and calculate the average value of the 'Rating' column
country_grouped = df.groupby('Country')['Rating'].mean().reset_index()

# Rename the column for clarity
country_grouped.columns = ['Country', 'Average_Rating']

print("Average Rating by Country:", "\n")
print(country_grouped)

# Equivalent PostgreSQL query:
"""     SELECT Country, AVG(Rating) AS Average_Rating
        FROM tourism_dataset
        GROUP BY Country;
"""

Average Rating by Country: 

     Country  Average_Rating
0  Australia        3.019602
1     Brazil        3.074167
2      China        2.958648
3      Egypt        3.024298
4     France        3.030268
5      India        2.973158
6        USA        2.984304


'     SELECT Country, AVG(Rating) AS Average_Rating\n        FROM tourism_dataset\n        GROUP BY Country;\n'

In [46]:

# Group the data by 'Category' and calculate the average value of the 'Rating' column
category_grouped = df.groupby('Category')['Rating'].mean().reset_index()

# Sort the categories by average rating in descending order and select the top 3
top_categories = category_grouped.sort_values(by='Rating', ascending=False).head(3)

print("\nTop 3 Categories with Highest Average Rating:", "\n")
print(top_categories)

# Equivalent PostgreSQL query:
"""     SELECT Category, AVG(Rating) AS Average_Rating
        FROM tourism_dataset
        GROUP BY Category
        ORDER BY Average_Rating DESC
        LIMIT 3;
"""



Top 3 Categories with Highest Average Rating: 

     Category    Rating
1       Beach  3.072285
0   Adventure  3.008804
3  Historical  3.003139


'     SELECT Category, AVG(Rating) AS Average_Rating\n        FROM tourism_dataset\n        GROUP BY Category\n        ORDER BY Average_Rating DESC\n        LIMIT 3;\n'

## Step 4: Export Results and Save to VM

In [50]:
with open('Saim-Nazir.csv', 'w') as f:
    f.write("Country Average Rates\n")
    country_grouped.to_csv(f, index=False)
    
    f.write("\n")

    f.write("Top 3 Categories by Average Rate\n")
    top_categories.to_csv(f, index=False)

In [56]:
container_name = "saim-nazir"
file_name = 'Saim-Nazir.csv'

#container_client = blob_service_client.create_container(name=container_name)

file_path = os.path.join(os.getcwd(), file_name) 

blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

with open(file_path, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

In [61]:
"""Bash commands to SSH into VM and download CSV file using Azure CLI"""

# https://learn.microsoft.com/en-us/azure/virtual-machines/linux-vm-connect?source=recommendations&tabs=Linux

#ssh-keygen 
#cd .ssh  

# Configure VM in Portal. Added private key path under VM-Saim | Connect > Native SSH

#sudo chmod 400 id_rsa    
#ssh -i id_rsa Saim@40.68.255.252


# curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
# az login
# az storage blob download  --account-name dataengineerv1 --account-key "" -container-name saim-nazir --blob-name result-Saim.csv     --file Saim-Nazir.csv

'Bash commands to SSH and download to VM'